In [1]:
import qiskit
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer,execute, transpile

### Idea:
##### Step 1: Take the given integer (say a_int) and represent it in a quantum circuit in binary form (say a).
##### Step 2: Implement a quantum circuit to take the first element of the list (say b) and find its two's complement form (which is equivalent to the negative of the number) (say b_tcomp).
##### Step 3: Implement a quantum circuit to add given integer (a) and two's complement of b (b_tcomp) from step2.  {Essentially I am just doing (a-b) arithmetics}
##### Step 4: On measuring the above quantum circuit, convert the resulting binary string to an integer (say sum). {Classical post-processing}
##### Step 5: Check if the resulting integer (sum) belongs to the circuit and it is not equal to the initial integer we took (b).
##### Step 6: If Step 5 is satisfied, print the answer = (b,sum). If Step 5 is not satisfied, repeat the algorithm by taking the next number in the list from Step 2.

##### The advantage of this algorithm is that it takes maximum of only 'length(prime_number_list)/2' number of steps to find the answer.
##### Note: This algorithm stops once it finds a set of answers and thus prints only one set of answers. Ex: a = 18 ; list = [1,3,5,7,11,13,15], we have multiple solutions here like: (3,15); (7,11) ; (15,3). But this algorithm just prints out the first answer i.e., (3,15).
##### This algorithm can be further modified a bit to find all the solutions very easily.

In [2]:
def find_the_prime_numbers(a_int,b_list):
    b_list = b_list
    a = bin(a_int)[2:][::-1]
    answer1 = 0
    answer2 = 0
    for i in range(len(b_list)):
        element_b = b_list[i]
        b = bin(element_b)[2:].zfill(len(a))[::-1]

        #Step 2: finding two's complement of b
        q1 = QuantumRegister(len(a))
        q2 = QuantumRegister(len(a))
        q3 = QuantumRegister(len(a)) #carry bits
        c = ClassicalRegister(len(a))
        qc = QuantumCircuit(q1,q2,q3,c)
        #encoding b
        for i in range(len(b)):
            if int(b[i])==1:
                qc.x(q1[i])
        qc.barrier()
        qc.x(q1[:])
        qc.x(q3[-1])
        qc.barrier()
        qc.x(q2[0])
        #Adder circuit to add binary bit 1 to b1
        #adding q1[0] and q2[0]
        qc.ccx(q1[0],q2[0],q3[0])
        qc.cx(q1[0],q2[0])
        qc.barrier()
        for i in range(1,len(b)):
            qc.ccx(q1[i],q3[i-1],q3[i])
            qc.cx(q1[i],q3[i-1])
            qc.ccx(q3[i-1],q2[i],q3[i])
            qc.cx(q3[i-1],q2[i])
            qc.barrier()
        qc.measure(q2[:],c[:])
        result = execute(qc,Aer.get_backend('qasm_simulator')).result()
        counts = result.get_counts().keys()
        b_tcomp = [(outcome) for outcome in counts][0]

        #Step 3:adding a and b_tcomp
        a = bin(a_int)[2:].zfill(len(b_tcomp))  #Rewriting 'a' to make its length equivalent to length of two's complement of 'b'
        q1 = QuantumRegister(len(a))
        q2 = QuantumRegister(len(b_tcomp))
        q3 = QuantumRegister(len(a))
        c = ClassicalRegister(len(a))
        qc1 = QuantumCircuit(q1,q2,q3,c)
        #encoding a
        for i in range(len(a)):
            if int(a[i])==1:
                qc1.x(q1[i])
        #encoding b_tcomp
        for i in range(len(b_tcomp)):
            if int(b_tcomp[i])==1:
                qc1.x(q2[i])
        #adding both
        qc1.ccx(q1[len(a)-1],q2[len(a)-1],q3[len(a)-1])
        qc1.cx(q1[len(a)-1],q2[len(a)-1])
        qc1.barrier()
        for i in range(2,len(a)+1):
            qc1.ccx(q1[len(a)-i],q3[len(a)-i+1],q3[len(a)-i])
            qc1.cx(q1[len(a)-i],q3[len(a)-i+1])
            qc1.ccx(q2[len(a)-i],q3[len(a)-i+1],q3[len(a)-i])
            qc1.cx(q3[len(a)-i+1],q2[len(a)-i])
            qc1.barrier()
        qc1.measure(q2[:],c[:])
        #print(qc1.depth())
        #result
        t_qc1 = transpile(qc1,Aer.get_backend('qasm_simulator'),optimization_level=2)
        result = execute(t_qc1,Aer.get_backend('qasm_simulator')).result()
        counts = result.get_counts().keys()
        sum_bin = [(outcome) for outcome in counts][0][::-1]
        sum = int(sum_bin,2)

        #Step 4,5:classical post-processing 
        new_b = [x for x in b_list if x!=element_b]
        for i in range(len(new_b)):
            if new_b[i]==sum:
                answer1 = element_b
                answer2 = sum
                return "{},{}".format(answer1,answer2)       #Prints the answers if they exist or else prints nothing.

In [3]:
a = 18
b = [1,3,5,7,11,13,15]
A = find_the_prime_numbers(a,b)
print(A)

3,15
